In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

Mounted at /content/drive


In [3]:
!pip install jsonlines

In [4]:
from transformers import AutoTokenizer, RemBertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google/rembert")
model = RemBertModel.from_pretrained("google/rembert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

sentencepiece.model:   0%|          | 0.00/4.70M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.30G [00:00<?, ?B/s]

In [5]:
def encode(sent):
    inputs = tokenizer(sent, return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    return last_hidden_states.detach().numpy()[0]

def encode_sentence(sent):
    enc = encode(sent)
    pooled_enc = sum(enc) / enc.shape[0]
    return pooled_enc

def encode_candidates(cands):
    return [encode_sentence(cand) for cand in cands]

In [6]:
import jsonlines
# BLEURT
bleurt_DEEN_PATH = drive_PATH + '/data-main/bleurt-DEEN.jsonl'
bleurt_DEEN = []
with jsonlines.open(bleurt_DEEN_PATH) as f:
    for line in f.iter():
        bleurt_DEEN.append(line)
# chrF++
chrf_DEEN_PATH = drive_PATH + '/data-main/chrf-DEEN.jsonl'
chrf_DEEN = []

with jsonlines.open(chrf_DEEN_PATH) as f:
    for line in f.iter():
        chrf_DEEN.append(line)

In [7]:
num_examples = 1000
assert(len(chrf_DEEN) == num_examples)
assert(len(bleurt_DEEN) == num_examples)


for i in range(num_examples):
  assert(chrf_DEEN[i]['src'] == bleurt_DEEN[i]['src'])
  assert(chrf_DEEN[i]['ref'] == bleurt_DEEN[i]['ref'])
  assert(chrf_DEEN[i]['candidates'] == bleurt_DEEN[i]['candidates'])

In [8]:
import tqdm
DEEN = []

for i in tqdm.tqdm(range(num_examples)):
  DEEN_dict = { 'src': chrf_DEEN[i]['src'], 'ref': chrf_DEEN[i]['ref'], 'candidates': chrf_DEEN[i]['candidates'], 'enc_candidates':encode_candidates(chrf_DEEN[i]['candidates']) }
  ### bleurt
  # cand scores
  DEEN_dict['BLEURT_cand_scores'] = bleurt_DEEN[i]['BLEURT_cand_scores']
  # mbr cand scores
  DEEN_dict['MBR_BLEURT_cand_scores'] = bleurt_DEEN[i]['MBR_BLEURT_cand_scores']
  # mbr score matrix
  DEEN_dict['MBR_BLEURT_score_matrix'] = bleurt_DEEN[i]['MBR_BLEURT_score_matrix']
  # mbr winner index
  DEEN_dict['MBR_BLEURT_winner_index'] = bleurt_DEEN[i]['MBR_BLEURT_winner_index']
  # mbr winner
  DEEN_dict['MBR_BLEURT_winner'] = bleurt_DEEN[i]['MBR_BLEURT_winner']
  ### chrf
  # cand scores
  DEEN_dict['CHRF_cand_scores'] = chrf_DEEN[i]['CHRF_cand_scores']
  # mbr cand scores
  DEEN_dict['MBR_CHRF_cand_scores'] = chrf_DEEN[i]['MBR_CHRF_cand_scores']
  # mbr score matrix
  DEEN_dict['MBR_CHRF_score_matrix'] = chrf_DEEN[i]['MBR_CHRF_score_matrix']
  # mbr winner index
  DEEN_dict['MBR_CHRF_winner_index'] = chrf_DEEN[i]['MBR_CHRF_winner_index']
  # mbr winner
  DEEN_dict['MBR_CHRF_winner'] = chrf_DEEN[i]['MBR_CHRF_winner']
  ###
  DEEN.append(DEEN_dict)

100%|██████████| 1000/1000 [8:54:07<00:00, 32.05s/it]


In [10]:
import numpy as np
for i in tqdm.tqdm(range(num_examples)):
  DEEN[i]['enc_candidates'] = np.array(DEEN[i]['enc_candidates'], dtype=float).tolist()

100%|██████████| 1000/1000 [00:06<00:00, 145.18it/s]


In [14]:
# DEEN[999].keys()

dict_keys(['src', 'ref', 'candidates', 'enc_candidates', 'BLEURT_cand_scores', 'MBR_BLEURT_cand_scores', 'MBR_BLEURT_score_matrix', 'MBR_BLEURT_winner_index', 'MBR_BLEURT_winner', 'CHRF_cand_scores', 'MBR_CHRF_cand_scores', 'MBR_CHRF_score_matrix', 'MBR_CHRF_winner_index', 'MBR_CHRF_winner'])

In [11]:
DEEN_PATH = drive_PATH+ f'/data-main/DEEN.jsonl'
with jsonlines.open(DEEN_PATH, mode='w') as writer:
    for item in DEEN:
        writer.write(item)